In [ ]:
import pandas as pd
import re
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('../input/commonlitreadabilityprize/train.csv')

In [ ]:
df = df.fillna('Missing')

In [ ]:
# df.target = df.target.apply(lambda x: np.floor(x))

In [ ]:
sns.boxplot(x=df.target)
plt.show()

In [ ]:
# df.target = df.target.apply(lambda x: -3 if (x==-4) else x)
# df.target = df.target.apply(lambda x: 0 if (x==1) else x)

In [ ]:
# sns.boxplot(x=df.target)
# plt.show()

In [ ]:
df['ex_len'] = df.excerpt.apply(lambda x: len(x))

sns.histplot(x=df.ex_len)
plt.show()

In [ ]:
df

In [ ]:
%%time

import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

def cleaner(excerpt):
    clean = nltk.word_tokenize(re.sub("[^a-zA-Z]", " ", excerpt).lower())
    clean = [word for word in clean if not word in set(stopwords.words("english"))]

    lem = nltk.WordNetLemmatizer()
    clean = [lem.lemmatize(word) for word in clean]
    return " ".join(clean)

df.excerpt = df.excerpt.apply(cleaner)

In [ ]:
wdf = pd.read_csv('../input/english-word-frequency/unigram_freq.csv')
wdf['ncol'] = wdf.word.apply(lambda x: True if (x not in set(stopwords.words("english"))) else False)
nwdf = wdf[wdf.ncol==True]

lem1 = nltk.WordNetLemmatizer()
nwdf['lword'] = nwdf.word.apply(lambda x: lem1.lemmatize(str(x)))
nwdf = nwdf.sort_values('count')

def change_scale_word_count(old_value):
    return ( (old_value - 12711) / (1551258643 - 12711) ) * (1 - 0) + 0

nwdf['scaled_count'] = nwdf['count'] #.apply(change_scale_word_count)
word_freq = dict(zip(nwdf.word, nwdf.scaled_count))

def get_score(excerpt):
    score = 0

    for i in excerpt.split(' '):
        try:
            score += word_freq[i]
        except KeyError:
            pass

    return score

In [ ]:
nwdf

In [ ]:
df['excerpt_score'] = df.excerpt.apply(get_score)

In [ ]:
arc_df = df.copy()

In [ ]:
df = arc_df.copy()

In [ ]:
df

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mms1 = MinMaxScaler()
mms2 = MinMaxScaler()
mms3 = MinMaxScaler()

# df.target = mms1.fit_transform(np.reshape(list(df.target), (-1,1)))
df.excerpt_score = mms2.fit_transform(np.reshape(list(df.excerpt_score), (-1,1)))
df.ex_len = mms3.fit_transform(np.reshape(list(df.ex_len), (-1,1)))


In [ ]:
y = df['target']
df.drop(['id', 'url_legal', 'license', 'target', 'standard_error'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
name_vectorizer = TfidfVectorizer()
names_encoded = name_vectorizer.fit_transform(df.excerpt)
print(names_encoded.shape)
names_df = pd.DataFrame(data=names_encoded.toarray(), columns=name_vectorizer.get_feature_names())
df = pd.concat([df, names_df], axis=1)

In [ ]:
df.drop(['excerpt'], axis=1, inplace=True)

In [ ]:
X = df

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse(y_test, y_pred)

In [ ]:
# from sklearn.svm import SVR
# model = SVR()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# mse(y_test, y_pred)

# # 0.0689411833459731

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse(y_test, y_pred)

# 0.05142008547008548

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor

# model = GradientBoostingRegressor()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# mse(y_test, y_pred)

# # 0.051231989788161326

In [ ]:
# from xgboost import XGBRegressor

# model = XGBRegressor()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# mse(y_test, y_pred)

# # 0.05074928177769481

### Test Data

In [ ]:
tdf = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
tdf = tdf.fillna('Missing')

tdf['ex_len'] = tdf.excerpt.apply(lambda x: len(x))
tdf.excerpt = tdf.excerpt.apply(cleaner)
tdf['excerpt_score'] = tdf.excerpt.apply(get_score)

tdf.drop(['id', 'url_legal', 'license'], axis=1, inplace=True)

In [ ]:
tdf

In [ ]:
names_encoded = name_vectorizer.transform(tdf.excerpt)
print(names_encoded.shape)
names_df = pd.DataFrame(data=names_encoded.toarray(), columns=name_vectorizer.get_feature_names())
tdf = pd.concat([tdf, names_df], axis=1)

In [ ]:
tdf.drop(['excerpt'], axis=1, inplace=True)

In [ ]:
tdf.excerpt_score = mms2.transform(np.reshape(list(tdf.excerpt_score), (-1,1)))
tdf.ex_len = mms3.transform(np.reshape(list(tdf.ex_len), (-1,1)))

In [ ]:
ypred = model.predict(tdf)

In [ ]:
ypred

In [ ]:
# nypred = mms1.inverse_transform(np.reshape(list(ypred), (-1,1)))[:, 0]

In [ ]:
nypred = ypred

In [ ]:
submission = pd.DataFrame({ 'id' : pd.read_csv('../input/commonlitreadabilityprize/test.csv')['id'], 
                           'target': list(nypred)})
submission.to_csv('submission.csv', index=False)
